In [1]:
import pandas as pd
import os
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import numpy as np

import xgboost as xgb

In [2]:
RANDOM_STATE = 1234
pd.options.display.precision = 20

In [3]:
DATA_DIR = 'data'

In [4]:
df_train = pd.read_csv(
    os.path.join(DATA_DIR, 'train.csv'),
    sep=','
)

df_test = pd.read_csv(
    os.path.join(DATA_DIR, 'test.csv'),
    sep=','
)

In [5]:
df_train.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,2c9180975a056a64015a1e0a52e57021,Rear,195,1.02000000000000001776,46,45,Rear
1,2c9180975a056a64015a1e17b32171e4,Rear,203,1.09000000000000007994,59,54,Rear
2,2c9180975a056a64015a1de4deb16bd5,Front,26,0.95999999999999996447,104,108,Front
3,2c9180975a056a64015a1de4deb16bdd,Rear,199,0.81000000000000005329,38,47,Rear
4,2c9180975a056a64015a1de4deb16bd6,Rear,208,0.93000000000000004885,54,58,Rear


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38485 entries, 0 to 38484
Data columns (total 7 columns):
Id                     38485 non-null object
DetectedCamera         38485 non-null object
AngleOfSign            38485 non-null int64
SignAspectRatio        38485 non-null float64
SignWidth              38485 non-null int64
SignHeight             38485 non-null int64
SignFacing (Target)    38485 non-null object
dtypes: float64(1), int64(3), object(3)
memory usage: 2.1+ MB


In [7]:
df_test.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,2c9180975a056a64015a1e10d3f270fe,Right,67,0.63000000000000000444,107,169
1,2c9180975a056a64015a1de4deb16bdc,Front,16,0.88000000000000000444,61,69
2,2c9180975a056a64015a1e0e70ea70ce,Right,44,1.14999999999999991118,232,202
3,2c9180975a056a64015a1dfed0c46ec6,Right,50,1.10000000000000008882,137,125
4,2c9180975a056a64015a1dfed0c46ec7,Front,30,0.94999999999999995559,99,104


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31485 entries, 0 to 31484
Data columns (total 6 columns):
Id                 31485 non-null object
DetectedCamera     31485 non-null object
AngleOfSign        31485 non-null int64
SignAspectRatio    31485 non-null float64
SignWidth          31485 non-null int64
SignHeight         31485 non-null int64
dtypes: float64(1), int64(3), object(2)
memory usage: 1.4+ MB


In [9]:
pd.crosstab(df_train['DetectedCamera'], df_train['SignFacing (Target)'], margins=True)

SignFacing (Target),Front,Left,Rear,Right,All
DetectedCamera,,,,,
Front,10759,101,0,50,10910
Left,1005,541,7752,0,9298
Rear,0,33,7657,71,7761
Right,9563,0,253,700,10516
All,21327,675,15662,821,38485


#### Feature Engineering

In [10]:
def get_sine(row):
    return np.sin(np.deg2rad(row['AngleOfSign']))

def get_cos(row):
    return np.cos(np.deg2rad(row['AngleOfSign']))

def get_tan(row):
    return np.tan(np.deg2rad(row['AngleOfSign']))

def convert_angle(row):
    if row['DetectedCamera'] == 'Front':
        return row['AngleOfSign']
    elif row['DetectedCamera'] == 'Left':
        return 270.0 - row['AngleOfSign']
    elif row['DetectedCamera'] == 'Right':
        return 90.0 - row['AngleOfSign']
    elif row['DetectedCamera'] == 'Rear':
        return 180.0 - row['AngleOfSign']

def add_trig_features(df):
    df.loc[
        :,
        'AngleOfSign'
    ] = df.apply(
        convert_angle,
        axis=1
    )
    
    df.loc[
        :,
        'sine_angle'
    ] = df.apply(
        get_sine,
        axis=1
    )
    
    df.loc[
        :,
        'cos_angle'
    ] = df.apply(
        get_cos,
        axis=1
    )
    
    df.loc[
        :,
        'tan_angle'
    ] = df.apply(
        get_tan,
        axis=1
    )
    
    return df

df_train = add_trig_features(df_train)
df_test = add_trig_features(df_test)

df_train.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target),sine_angle,cos_angle,tan_angle
0,2c9180975a056a64015a1e0a52e57021,Rear,-15.0,1.02000000000000001776,46,45,Rear,-0.25881904510252073948,0.96592582628906831221,-0.26794919243112269580
1,2c9180975a056a64015a1e17b32171e4,Rear,-23.0,1.09000000000000007994,59,54,Rear,-0.39073112848927377128,0.92050485345244037472,-0.42447481620960475501
2,2c9180975a056a64015a1de4deb16bd5,Front,26.0,0.95999999999999996447,104,108,Front,0.43837114678907740384,0.89879404629916703762,0.48773258856586138021
3,2c9180975a056a64015a1de4deb16bdd,Rear,-19.0,0.81000000000000005329,38,47,Rear,-0.32556815445715669988,0.94551857559931684616,-0.34432761328966526859
4,2c9180975a056a64015a1de4deb16bd6,Rear,-28.0,0.93000000000000004885,54,58,Rear,-0.46947156278589080580,0.88294759285892698841,-0.53170943166147877079


In [11]:
df_test.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,sine_angle,cos_angle,tan_angle
0,2c9180975a056a64015a1e10d3f270fe,Right,23.0,0.63000000000000000444,107,169,0.39073112848927377128,0.92050485345244037472,0.42447481620960475501
1,2c9180975a056a64015a1de4deb16bdc,Front,16.0,0.88000000000000000444,61,69,0.27563735581699916333,0.96126169593831889415,0.28674538575880792468
2,2c9180975a056a64015a1e0e70ea70ce,Right,46.0,1.14999999999999991118,232,202,0.71933980033865108084,0.69465837045899736513,1.03553031379056936778
3,2c9180975a056a64015a1dfed0c46ec6,Right,40.0,1.10000000000000008882,137,125,0.64278760968653925190,0.76604444311897812447,0.83909963117727981441
4,2c9180975a056a64015a1dfed0c46ec7,Front,30.0,0.94999999999999995559,99,104,0.49999999999999994449,0.86602540378443870761,0.57735026918962562004


#### Label Encoding

In [12]:
def label_encoder(df):
    for column in df.columns:
        if column == 'Id':
            continue
        if df[column].dtype == 'object':
            label = preprocessing.LabelEncoder()
            label.fit(list(df[column].values))
            df[column + str('_encoded')] = label.transform(list(df[column].values))
    return df

In [13]:
df_train['SignFacing (Target)'].value_counts()

Front    21327
Rear     15662
Right      821
Left       675
Name: SignFacing (Target), dtype: int64

In [14]:
df_train['DetectedCamera'].value_counts()

Front    10910
Right    10516
Left      9298
Rear      7761
Name: DetectedCamera, dtype: int64

In [15]:
df_test.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,sine_angle,cos_angle,tan_angle
0,2c9180975a056a64015a1e10d3f270fe,Right,23.0,0.63000000000000000444,107,169,0.39073112848927377128,0.92050485345244037472,0.42447481620960475501
1,2c9180975a056a64015a1de4deb16bdc,Front,16.0,0.88000000000000000444,61,69,0.27563735581699916333,0.96126169593831889415,0.28674538575880792468
2,2c9180975a056a64015a1e0e70ea70ce,Right,46.0,1.14999999999999991118,232,202,0.71933980033865108084,0.69465837045899736513,1.03553031379056936778
3,2c9180975a056a64015a1dfed0c46ec6,Right,40.0,1.10000000000000008882,137,125,0.64278760968653925190,0.76604444311897812447,0.83909963117727981441
4,2c9180975a056a64015a1dfed0c46ec7,Front,30.0,0.94999999999999995559,99,104,0.49999999999999994449,0.86602540378443870761,0.57735026918962562004


#### Let's build direction based models

In [16]:
df_train_front = df_train[df_train['DetectedCamera'] == 'Front']
df_train_left = df_train[df_train['DetectedCamera'] == 'Left']
df_train_rear = df_train[df_train['DetectedCamera'] == 'Rear']
df_train_right = df_train[df_train['DetectedCamera'] == 'Right']

df_train_front = label_encoder(df_train_front)
df_train_left = label_encoder(df_train_left)
df_train_rear = label_encoder(df_train_rear)
df_train_right = label_encoder(df_train_right)

df_train_front = df_train_front.reset_index()
df_train_left = df_train_left.reset_index()
df_train_rear = df_train_rear.reset_index()
df_train_right = df_train_right.reset_index()

df_test = label_encoder(df_test)

/Users/manrajsingh/virtualenvs/ds/lib/python2.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
df_train_front.head()

,index,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target),sine_angle,cos_angle,tan_angle,DetectedCamera_encoded,SignFacing (Target)_encoded
0,2,2c9180975a056a64015a1de4deb16bd5,Front,26.0,0.95999999999999996447,104,108,Front,0.43837114678907740384,0.89879404629916703762,0.48773258856586138021,0,0
1,5,2c9180975a056a64015a1e44ba4377f6,Front,29.0,0.82999999999999996003,65,78,Front,0.48480962024633705854,0.87461970713939574118,0.55430905145276898605,0,0
2,8,2c9180975a056a64015a1e01fd466f4f,Front,25.0,0.94999999999999995559,104,109,Front,0.42261826174069944129,0.90630778703664993667,0.46630765815499863880,0,0
3,10,2c9180975a056a64015a1e1db63072a6,Front,12.0,0.93000000000000004885,41,44,Front,0.20791169081775934258,0.97814760073380568883,0.21255656167002212609,0,0
4,15,2c9180975a056a64015a1df8dc786e26,Front,21.0,1.00000000000000000000,75,75,Front,0.35836794954530026835,0.93358042649720174300,0.38386403503541577376,0,0


In [18]:
df_train_left.head()

,index,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target),sine_angle,cos_angle,tan_angle,DetectedCamera_encoded,SignFacing (Target)_encoded
0,6,2c9180975a056a64015a1e01fd466f56,Left,25.0,0.93000000000000004885,109,117,Rear,0.42261826174069944129,0.90630778703664993667,0.46630765815499863880,0,2
1,14,2c9180975a056a64015a1e04ad9f6f8b,Left,12.0,0.47999999999999998224,39,81,Rear,0.20791169081775934258,0.97814760073380568883,0.21255656167002212609,0,2
2,29,2c9180975a056a64015a1e41bbc2775f,Left,17.0,0.68999999999999994671,37,54,Rear,0.29237170472273676936,0.95630475596303543551,0.30573068145866039158,0,2
3,30,2c9180975a056a64015a1e41bbc27758,Left,22.0,0.75000000000000000000,44,59,Rear,0.37460659341591201477,0.92718385456678742429,0.40402622583515679366,0,2
4,33,2c9180975a056a64015a1e13b6c7713e,Left,17.0,0.42999999999999999334,37,87,Rear,0.29237170472273676936,0.95630475596303543551,0.30573068145866039158,0,2


In [19]:
df_train_rear.head()

,index,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target),sine_angle,cos_angle,tan_angle,DetectedCamera_encoded,SignFacing (Target)_encoded
0,0,2c9180975a056a64015a1e0a52e57021,Rear,-15.0,1.02000000000000001776,46,45,Rear,-0.25881904510252073948,0.96592582628906831221,-0.26794919243112269580,0,1
1,1,2c9180975a056a64015a1e17b32171e4,Rear,-23.0,1.09000000000000007994,59,54,Rear,-0.39073112848927377128,0.92050485345244037472,-0.42447481620960475501,0,1
2,3,2c9180975a056a64015a1de4deb16bdd,Rear,-19.0,0.81000000000000005329,38,47,Rear,-0.32556815445715669988,0.94551857559931684616,-0.34432761328966526859,0,1
3,4,2c9180975a056a64015a1de4deb16bd6,Rear,-28.0,0.93000000000000004885,54,58,Rear,-0.46947156278589080580,0.88294759285892698841,-0.53170943166147877079,0,1
4,7,2c9180975a056a64015a1e01fd466f50,Rear,-24.0,1.04000000000000003553,56,54,Rear,-0.40673664307580020827,0.91354545764260086660,-0.44522868530853615399,0,1


In [20]:
df_train_right.head()

,index,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target),sine_angle,cos_angle,tan_angle,DetectedCamera_encoded,SignFacing (Target)_encoded
0,9,2c9180975a056a64015a1e01fd466f51,Right,16.0,0.51000000000000000888,53,103,Front,0.27563735581699916333,0.96126169593831889415,0.28674538575880792468,0,0
1,21,2c9180975a056a64015a1e04ad9f6f83,Right,41.0,1.04000000000000003553,127,122,Front,0.65605902899050727584,0.75470958022277201405,0.86928673781622667338,0,0
2,27,2c9180975a056a64015a1e1d735c7293,Right,14.0,0.64000000000000001332,65,102,Front,0.24192189559966773005,0.97029572627599647294,0.24932800284318071271,0,0
3,31,2c9180975a056a64015a1e41bbc27756,Right,18.0,0.55000000000000004441,52,94,Front,0.30901699437494739575,0.95105651629515353118,0.32491969623290628544,0,0
4,37,2c9180975a056a64015a1dceefde69de,Right,47.0,1.19999999999999995559,200,167,Front,0.73135370161917045984,0.68199836006249847653,1.07236871002468259206,0,0


In [21]:
pd.crosstab(df_train_front['DetectedCamera'], df_train_front['SignFacing (Target)'], margins=True)

SignFacing (Target),Front,Left,Right,All
DetectedCamera,,,,
Front,10759,101,50,10910
All,10759,101,50,10910


In [22]:
pd.crosstab(df_train_left['DetectedCamera'], df_train_left['SignFacing (Target)'], margins=True)

SignFacing (Target),Front,Left,Rear,All
DetectedCamera,,,,
Left,1005,541,7752,9298
All,1005,541,7752,9298


In [23]:
pd.crosstab(df_train_rear['DetectedCamera'], df_train_rear['SignFacing (Target)'], margins=True)

SignFacing (Target),Left,Rear,Right,All
DetectedCamera,,,,
Rear,33,7657,71,7761
All,33,7657,71,7761


In [24]:
pd.crosstab(df_train_right['DetectedCamera'], df_train_right['SignFacing (Target)'], margins=True)

SignFacing (Target),Front,Rear,Right,All
DetectedCamera,,,,
Right,9563,253,700,10516
All,9563,253,700,10516


In [25]:
df_test[df_test['Id'] == '2c9180975a056a64015a1e10d3f270fe']

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,sine_angle,cos_angle,tan_angle,DetectedCamera_encoded
0,2c9180975a056a64015a1e10d3f270fe,Right,23.0,0.63000000000000000444,107,169,0.39073112848927377128,0.92050485345244037472,0.42447481620960475501,3


In [26]:
req_cols = [
    'sine_angle',
    'cos_angle',
    'tan_angle',
    'Id',
    'AngleOfSign',
    'SignAspectRatio',
    'SignWidth',
    'SignHeight',
    'DetectedCamera_encoded',
    'SignFacing (Target)_encoded'
]

def get_train_test(df):
    df = df[req_cols]
    X = df.drop(['SignFacing (Target)_encoded', 'Id'], axis=1)
    y = df['SignFacing (Target)_encoded']

    return train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

def train_model(X_train, X_test, y_train, y_test):
    params = {
        'n_estimators': 5000,
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        'colsample_bytree': 0.75,
        'min_child_weight': 4,
        'subsample': 0.8,
        'eta': 0.01,
        'max_depth': 4,
        'gamma': 0,
        'seed': 1,
        'num_class': 3
    }

    d_train = xgb.DMatrix(X_train, label=y_train)
    d_test = xgb.DMatrix(X_test, label=y_test)

    watchlist = [(d_train, 'train'), (d_test, 'valid')]

    model = xgb.train(params, d_train, 2000, watchlist, early_stopping_rounds=50, verbose_eval=10)
    
    return model

##### Front Camera

In [27]:
df_train_front.columns

Index([u'index', u'Id', u'DetectedCamera', u'AngleOfSign', u'SignAspectRatio',
       u'SignWidth', u'SignHeight', u'SignFacing (Target)', u'sine_angle',
       u'cos_angle', u'tan_angle', u'DetectedCamera_encoded',
       u'SignFacing (Target)_encoded'],
      dtype='object')

In [28]:
df_train_front = df_train_front[req_cols]
df_train_front.head()

,sine_angle,cos_angle,tan_angle,Id,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,DetectedCamera_encoded,SignFacing (Target)_encoded
0,0.43837114678907740384,0.89879404629916703762,0.48773258856586138021,2c9180975a056a64015a1de4deb16bd5,26.0,0.95999999999999996447,104,108,0,0
1,0.48480962024633705854,0.87461970713939574118,0.55430905145276898605,2c9180975a056a64015a1e44ba4377f6,29.0,0.82999999999999996003,65,78,0,0
2,0.42261826174069944129,0.90630778703664993667,0.46630765815499863880,2c9180975a056a64015a1e01fd466f4f,25.0,0.94999999999999995559,104,109,0,0
3,0.20791169081775934258,0.97814760073380568883,0.21255656167002212609,2c9180975a056a64015a1e1db63072a6,12.0,0.93000000000000004885,41,44,0,0
4,0.35836794954530026835,0.93358042649720174300,0.38386403503541577376,2c9180975a056a64015a1df8dc786e26,21.0,1.00000000000000000000,75,75,0,0


In [29]:
df_train_front = df_train_front[req_cols]
X_train, X_test, y_train, y_test = get_train_test(df_train_front)
X_train.head()

,sine_angle,cos_angle,tan_angle,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,DetectedCamera_encoded
3976,0.32556815445715669988,0.94551857559931684616,0.34432761328966526859,19.0,0.83999999999999996891,42,50,0
3663,0.17364817766693033119,0.98480775301220802032,0.17632698070846497540,10.0,0.93999999999999994671,49,52,0
8251,0.46947156278589080580,0.88294759285892698841,0.53170943166147877079,28.0,0.92000000000000003997,152,165,0
5406,0.39073112848927377128,0.92050485345244037472,0.42447481620960475501,23.0,0.95999999999999996447,65,68,0
8911,0.39073112848927377128,0.92050485345244037472,0.42447481620960475501,23.0,1.04000000000000003553,227,218,0


In [30]:
model_front = train_model(X_train, X_test, y_train, y_test)

[0]	train-mlogloss:1.08409	valid-mlogloss:1.08425
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[10]	train-mlogloss:0.952353	valid-mlogloss:0.953888
[20]	train-mlogloss:0.841302	valid-mlogloss:0.844033
[30]	train-mlogloss:0.746535	valid-mlogloss:0.750362
[40]	train-mlogloss:0.6649	valid-mlogloss:0.669783
[50]	train-mlogloss:0.594063	valid-mlogloss:0.599908
[60]	train-mlogloss:0.532225	valid-mlogloss:0.538998
[70]	train-mlogloss:0.478005	valid-mlogloss:0.485557
[80]	train-mlogloss:0.430232	valid-mlogloss:0.438636
[90]	train-mlogloss:0.388085	valid-mlogloss:0.397184
[100]	train-mlogloss:0.350731	valid-mlogloss:0.360513
[110]	train-mlogloss:0.317549	valid-mlogloss:0.327929
[120]	train-mlogloss:0.288026	valid-mlogloss:0.299056
[130]	train-mlogloss:0.261697	valid-mlogloss:0.273325
[140]	train-mlogloss:0.238207	valid-mlogloss:0.250367
[150]	train-mlogloss:0.217185	valid-mlogloss:0.22989

#### Left Camera

In [31]:
df_train_left = df_train_left[req_cols]
X_train, X_test, y_train, y_test = get_train_test(df_train_left)
model_left = train_model(X_train, X_test, y_train, y_test)

[0]	train-mlogloss:1.08587	valid-mlogloss:1.08599
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[10]	train-mlogloss:0.970986	valid-mlogloss:0.971843
[20]	train-mlogloss:0.874233	valid-mlogloss:0.875692
[30]	train-mlogloss:0.791166	valid-mlogloss:0.793301
[40]	train-mlogloss:0.719532	valid-mlogloss:0.722192
[50]	train-mlogloss:0.657459	valid-mlogloss:0.660773
[60]	train-mlogloss:0.603221	valid-mlogloss:0.607069
[70]	train-mlogloss:0.555613	valid-mlogloss:0.560054
[80]	train-mlogloss:0.513764	valid-mlogloss:0.518643
[90]	train-mlogloss:0.476577	valid-mlogloss:0.48202
[100]	train-mlogloss:0.443845	valid-mlogloss:0.449697
[110]	train-mlogloss:0.414824	valid-mlogloss:0.421076
[120]	train-mlogloss:0.389008	valid-mlogloss:0.395693
[130]	train-mlogloss:0.365978	valid-mlogloss:0.373145
[140]	train-mlogloss:0.345357	valid-mlogloss:0.352986
[150]	train-mlogloss:0.327015	valid-mlogloss:0.3351

#### Rear Camera

In [32]:
df_train_rear = df_train_rear[req_cols]
X_train, X_test, y_train, y_test = get_train_test(df_train_rear)
model_rear = train_model(X_train, X_test, y_train, y_test)

[0]	train-mlogloss:1.0841	valid-mlogloss:1.08405
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[10]	train-mlogloss:0.952652	valid-mlogloss:0.952241
[20]	train-mlogloss:0.841823	valid-mlogloss:0.841087
[30]	train-mlogloss:0.747243	valid-mlogloss:0.746227
[40]	train-mlogloss:0.66575	valid-mlogloss:0.664544
[50]	train-mlogloss:0.595094	valid-mlogloss:0.593713
[60]	train-mlogloss:0.533399	valid-mlogloss:0.531838
[70]	train-mlogloss:0.479298	valid-mlogloss:0.477517
[80]	train-mlogloss:0.431728	valid-mlogloss:0.429792
[90]	train-mlogloss:0.389696	valid-mlogloss:0.387655
[100]	train-mlogloss:0.35248	valid-mlogloss:0.350332
[110]	train-mlogloss:0.319475	valid-mlogloss:0.31721
[120]	train-mlogloss:0.290089	valid-mlogloss:0.287719
[130]	train-mlogloss:0.263882	valid-mlogloss:0.261469
[140]	train-mlogloss:0.240504	valid-mlogloss:0.238048
[150]	train-mlogloss:0.219626	valid-mlogloss:0.217092


#### Right Camera

In [33]:
df_train_right = df_train_right[req_cols]
X_train, X_test, y_train, y_test = get_train_test(df_train_right)
model_right = train_model(X_train, X_test, y_train, y_test)

[0]	train-mlogloss:1.08526	valid-mlogloss:1.08511
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 50 rounds.
[10]	train-mlogloss:0.964604	valid-mlogloss:0.963372
[20]	train-mlogloss:0.86277	valid-mlogloss:0.860855
[30]	train-mlogloss:0.775704	valid-mlogloss:0.773175
[40]	train-mlogloss:0.700735	valid-mlogloss:0.697715
[50]	train-mlogloss:0.635548	valid-mlogloss:0.632151
[60]	train-mlogloss:0.578525	valid-mlogloss:0.574871
[70]	train-mlogloss:0.52854	valid-mlogloss:0.524759
[80]	train-mlogloss:0.484533	valid-mlogloss:0.48069
[90]	train-mlogloss:0.445581	valid-mlogloss:0.441836
[100]	train-mlogloss:0.411053	valid-mlogloss:0.407431
[110]	train-mlogloss:0.380355	valid-mlogloss:0.376861
[120]	train-mlogloss:0.353066	valid-mlogloss:0.349711
[130]	train-mlogloss:0.328548	valid-mlogloss:0.32549
[140]	train-mlogloss:0.306692	valid-mlogloss:0.303886
[150]	train-mlogloss:0.287159	valid-mlogloss:0.284612


In [34]:
df_test[df_test['Id'] == '2c9180975a056a64015a1e10d3f270fe']

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,sine_angle,cos_angle,tan_angle,DetectedCamera_encoded
0,2c9180975a056a64015a1e10d3f270fe,Right,23.0,0.63000000000000000444,107,169,0.39073112848927377128,0.92050485345244037472,0.42447481620960475501,3


In [35]:
features = req_cols
features.remove('SignFacing (Target)_encoded')
features.remove('Id')

results = pd.DataFrame()

results['Id'] = df_test['Id']
results['Front'] = None
results['Left'] = None
results['Rear'] = None
results['Right'] = None


def get_prediction(row):
    id = row['Id']
    df_row = df_test[df_test['Id'] == id]
    dtest = xgb.DMatrix(df_row[features])
    
    detecting_camera = df_row['DetectedCamera_encoded'].values[0]
    
    if detecting_camera == 0:
        y_pred = model_front.predict(dtest)[0]
        row['Front'] = y_pred[0]
        row['Left'] = y_pred[1]
        row['Rear'] = 0.0
        row['Right'] = y_pred[2]
    elif detecting_camera == 1:
        y_pred = model_left.predict(dtest)[0]
        row['Front'] = y_pred[0]
        row['Left'] = y_pred[1]
        row['Rear'] = y_pred[2]
        row['Right'] = 0.0
    elif detecting_camera == 2:
        y_pred = model_rear.predict(dtest)[0]
        row['Front'] = 0
        row['Left'] = y_pred[0]
        row['Rear'] = y_pred[1]
        row['Right'] = y_pred[2]
    elif detecting_camera == 3:
        y_pred = model_right.predict(dtest)[0]
        row['Front'] = y_pred[0]
        row['Left'] = 0
        row['Rear'] = y_pred[1]
        row['Right'] = y_pred[2]
    
    return row

df_test.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,sine_angle,cos_angle,tan_angle,DetectedCamera_encoded
0,2c9180975a056a64015a1e10d3f270fe,Right,23.0,0.63000000000000000444,107,169,0.39073112848927377128,0.92050485345244037472,0.42447481620960475501,3
1,2c9180975a056a64015a1de4deb16bdc,Front,16.0,0.88000000000000000444,61,69,0.27563735581699916333,0.96126169593831889415,0.28674538575880792468,0
2,2c9180975a056a64015a1e0e70ea70ce,Right,46.0,1.14999999999999991118,232,202,0.71933980033865108084,0.69465837045899736513,1.03553031379056936778,3
3,2c9180975a056a64015a1dfed0c46ec6,Right,40.0,1.10000000000000008882,137,125,0.64278760968653925190,0.76604444311897812447,0.83909963117727981441,3
4,2c9180975a056a64015a1dfed0c46ec7,Front,30.0,0.94999999999999995559,99,104,0.49999999999999994449,0.86602540378443870761,0.57735026918962562004,0


In [36]:
df_test[df_test['Id'] == '2c9180975a056a64015a1e10d3f270fe']

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,sine_angle,cos_angle,tan_angle,DetectedCamera_encoded
0,2c9180975a056a64015a1e10d3f270fe,Right,23.0,0.63000000000000000444,107,169,0.39073112848927377128,0.92050485345244037472,0.42447481620960475501,3


In [37]:
df_test.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,sine_angle,cos_angle,tan_angle,DetectedCamera_encoded
0,2c9180975a056a64015a1e10d3f270fe,Right,23.0,0.63000000000000000444,107,169,0.39073112848927377128,0.92050485345244037472,0.42447481620960475501,3
1,2c9180975a056a64015a1de4deb16bdc,Front,16.0,0.88000000000000000444,61,69,0.27563735581699916333,0.96126169593831889415,0.28674538575880792468,0
2,2c9180975a056a64015a1e0e70ea70ce,Right,46.0,1.14999999999999991118,232,202,0.71933980033865108084,0.69465837045899736513,1.03553031379056936778,3
3,2c9180975a056a64015a1dfed0c46ec6,Right,40.0,1.10000000000000008882,137,125,0.64278760968653925190,0.76604444311897812447,0.83909963117727981441,3
4,2c9180975a056a64015a1dfed0c46ec7,Front,30.0,0.94999999999999995559,99,104,0.49999999999999994449,0.86602540378443870761,0.57735026918962562004,0


In [38]:
results.head()

,Id,Front,Left,Rear,Right
0,2c9180975a056a64015a1e10d3f270fe,None,None,None,None
1,2c9180975a056a64015a1de4deb16bdc,None,None,None,None
2,2c9180975a056a64015a1e0e70ea70ce,None,None,None,None
3,2c9180975a056a64015a1dfed0c46ec6,None,None,None,None
4,2c9180975a056a64015a1dfed0c46ec7,None,None,None,None


In [39]:
results = results.apply(
    get_prediction,
    axis=1
)

In [40]:
results.head(n=20)

,Id,Front,Left,Rear,Right
0,2c9180975a056a64015a1e10d3f270fe,0.99606090784072875977,0.00000000000000000000,0.00211428594775497913,0.00182480423245579004
1,2c9180975a056a64015a1de4deb16bdc,0.99766552448272705078,0.00113862717989832163,0.00000000000000000000,0.00119586847722530365
2,2c9180975a056a64015a1e0e70ea70ce,0.99417746067047119141,0.00000000000000000000,0.00408017775043845177,0.00174234551377594471
3,2c9180975a056a64015a1dfed0c46ec6,0.99710339307785034180,0.00000000000000000000,0.00170045834966003895,0.00119611085392534733
4,2c9180975a056a64015a1dfed0c46ec7,0.99782866239547729492,0.00108278403058648109,0.00000000000000000000,0.00108857441227883101
5,2c9180975a056a64015a1e44ba4377f8,0.03375058993697166443,0.00000000000000000000,0.81827080249786376953,0.14797858893871307373
6,2c9180975a056a64015a1e010c8f6ee1,0.99823617935180664062,0.00089428329374641180,0.00000000000000000000,0.00086946430383250117
7,2c9180975a056a64015a1e1db63072aa,0.99413001537322998047,0.00000000000000000000,0.00351654784753918648,0.00235342839732766151
8,2c9180975a056a64015a1e1db63072a8,0.00000000000000000000,0.00093884282978251576,0.99835336208343505859,0.00070779671659693122
9,2c9180975a056a64015a1e2354f5733c,0.00000000000000000000,0.00095122592756524682,0.99831342697143554688,0.00073533930117264390


In [41]:
results.to_csv(os.path.join(DATA_DIR, 'predict_road_sign_30_06_2017_4_directions_fe.csv'), index=False, float_format='%0.10f')